In [ ]:
#1 client
import xmlrpc.client
proxy=xmlrpc.client.ServerProxy("http://localhost:8000")
n=int(input("enter the non-negative number for calculate factorial"))
result=proxy.compute_factorial(n)
print(f"factorial of {n} is {result}")

#server
from xmlrpc.server import SimpleXMLRPCServer 
def factorial(n):
    if(n==1):
        return 1
    return n*factorial(n-1)
server= SimpleXMLRPCServer(("localhost",8000))
print ("server is running in localhost:8000")
server.register_function(factorial,"compute_factorial")
server.serve_forever()
'''
 XML-RPC is a protocol that allows communication between a client and server using remote procedure calls encoded in XML.
 The client sends a number to the server and calls the compute_factorial method remotely to get the factorial result.
The server defines a factorial function, registers it for remote access, and listens on localhost:8000.
SimpleXMLRPCServer from the xmlrpc.server module.
The server calculates 5*4*3*2*1 = 120 and sends 120 back to the client.
 If n == 1, it returns 1.
 By calling server.serve_forever() after registering the function.'''

In [ ]:
#2 client
import Pyro5.api
uri=input("Enter the server url:")
server=Pyro5.api.Proxy(uri)
a=input("enter first string")
b=input("enter second string")
result=server.concat(a,b)
print("result:",result)

#server
import Pyro5.api
@Pyro5.api.expose
class Concatinate:
    def concat(self,a,b):
        return a+b
daemon=Pyro5.api.Daemon()
uri=daemon.register(Concatinate)
print("server is ready: ",uri)
daemon.requestLoop()
'''In this program, Pyro5 (Python Remote Objects) is used to enable remote method invocation between a client and server.
The server defines a class Concatinate with a method concat that takes two strings and returns their concatenation.
The @Pyro5.api.expose decorator is used to make the class's methods available for remote access. 
The server creates a Daemon, registers the class with it, and continuously listens for client requests using requestLoop(). 
On the client side, the program asks for the server's URI (Uniform Resource Identifier) to connect, then takes two string inputs from the user.
It sends these strings to the server's concat method and prints the result. 
This practical demonstrates how remote objects can be accessed and invoked easily over a network using Pyro5, simplifying distributed computing.'''

In [ ]:
#4 fuzzy operation
import numpy as np
# Fuzzy sets
A = np.array([0.3, 0.7, 1.0])
B = np.array([0.6, 0.5, 0.2])
# Union
print("Union:", np.maximum(A, B))
# Intersection
print("Intersection:", np.minimum(A, B))
# Complement
print("Complement of A:", 1 - A)
# Difference
print("Difference A - B:", np.minimum(A, 1 - B))
# Cartesian Product
relation1 = np.minimum(A[:, None], B[None, :])
relation2 = np.minimum(B[:, None], A[None, :])
print("Relation (A x B):\n", relation1)
print("Relation (B x A):\n", relation2)
# Max-Min Composition
comp = np.zeros((relation1.shape[0], relation2.shape[1]))
for i in range(comp.shape[0]):
    for j in range(comp.shape[1]):
        comp[i, j] = np.max(np.minimum(relation1[i, :], relation2[:, j]))
print("Max-Min Composition:\n",comp)
'''This program demonstrates basic operations on fuzzy sets using NumPy in Python. Two fuzzy sets, A and B, are represented as arrays with membership values between 0 and 1.
The union of A and B is calculated by taking the element-wise maximum, while the intersection is computed by taking the element-wise minimum. 
The complement of set A is found by subtracting each membership value from 1. 
The difference between A and B is evaluated by taking the minimum of each element in A and the complement of the corresponding element in B. 
The Cartesian product of A and B is constructed using the minimum of pairwise elements, forming a relation matrix. Additionally, the Max-Min composition is performed, which finds the maximum of the minimum values across relations to combine two fuzzy relations. This practical provides a foundational understanding of operations commonly used in fuzzy logic and fuzzy set theory.'''

In [ ]:
#5 Load balancing using round robin
class Server:
    def __init__(self,server_id):
        self.server_id=server_id
        self.connection=0
    def handle_request(self):
        self.connection+=1
        return self.connection-1

class Load_balacing:
    def __init__(self,servers):
        self.servers=servers
    def round_robin(self,request_id):
        sv=self.servers.pop(0)
        sv.handle_request()
        print(f"request:{request_id} ->servers:{sv.server_id}")
        self.servers.append(sv)    
    def get_load(self):
        return {sv.server_id: sv.handle_request() for sv in self.servers}

    sv = [Server(i) for i in range(3)]
    lb = Load_balacing(sv)
    # Distribute requests
    for req_id in range(1, 11):
        lb.round_robin(req_id)
    print("\nFinal server loads:")
    lb.get_load()

'''👉 Operations and Explanation:
Union: Take maximum values element-wise → np.maximum(A, B).
Intersection: Take minimum values element-wise → np.minimum(A, B).
Complement: Subtract A’s values from 1 → 1 - A.
Difference (A - B): Take minimum between A and complement of B → np.minimum(A, 1 - B).
Cartesian Product: Create a matrix where each element is min(A[i], B[j]).
Max-Min Composition: Combine two relations by taking the max of min values across rows and columns.'''

In [ ]:
#6 Clonal selection
import random

# Objective function to minimize (e.g., find minimum of x^2 + y^2)
def objective(x, y):
    return x**2 + y**2

# Initialize random population
def init_population(size, bounds):
    return [[random.uniform(*bounds), random.uniform(*bounds)] for _ in range(size)]

# Select the best antibodies (solutions)
def select_best(population, size):
    population.sort(key=lambda x: objective(*x))  # Sort by objective value
    return population[:size]

# Clone and mutate antibodies
def clone_and_mutate(population, clone_factor, mutation_rate, bounds):
    clones = []
    for p in population:
        for _ in range(clone_factor):
            clone = [p[0] + random.uniform(-0.1, 0.1), p[1] + random.uniform(-0.1, 0.1)]
            if random.random() < mutation_rate:
                clone[0] = random.uniform(*bounds)  # Mutate randomly
                clone[1] = random.uniform(*bounds)
            clones.append(clone)
    return clones

# Clonal Selection Algorithm
def clonal_selection(pop_size, generations, clone_factor, mutation_rate, bounds):
    population = init_population(pop_size, bounds)
    for gen in range(generations):
        best = select_best(population, pop_size // 2)
        clones = clone_and_mutate(best, clone_factor, mutation_rate, bounds)
        population = best + clones
        best_solution = min(population, key=lambda x: objective(*x))
        print(f"Gen {gen+1}: Best Solution: {best_solution}, Objective: {objective(*best_solution)}")
    return min(population, key=lambda x: objective(*x))

# Parameters
pop_size = 10
generations = 50 
clone_factor = 3 
mutation_rate = 0.1
bounds = (-5, 5)

# Run the algorithm
best = clonal_selection(pop_size, generations, clone_factor, mutation_rate, bounds)
print(f"Best Solution Found: {best}, Objective: {objective(*best)}")
'''This program uses a Clonal Selection Algorithm (based on immune system behavior) to find the minimum of a given function, here x² + y².
👉 Steps:
Initialization: Randomly create a starting population (antibodies/solutions).
Selection: Choose the best half of the population (lowest objective value).
Cloning and Mutation: Clone the best solutions multiple times and slightly mutate them to explore new possibilities.
Replacement: Combine the best and mutated clones to form the new generation.
Repeat: Run this process for several generations to gradually reach the optimal solution.
👉 Purpose:
To optimize a function using ideas inspired by the natural immune system, where better antibodies are cloned and mutated to find even better solutions.'''

In [ ]:
#7 immune patttern
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
# AIRS algorithm using SVM
def train_airs(X, y, detectors=20):
    indices = np.random.choice(len(X), detectors, replace=False)
    svm = SVC(kernel='rbf', probability=True)
    svm.fit(X[indices], y[indices])
    return svm
# Generate dummy data
data = np.random.rand(100, 10)
labels = np.random.randint(0, 2, 100)
# Split and scale data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train, X_test = scaler.fit_transform(X_train), scaler.transform(X_test)
# Train and evaluate
model = train_airs(X_train, y_train)
accuracy = (model.predict(X_test) == y_test).mean() * 100
print(f"Accuracy: {accuracy:.2f}%")

'''Purpose:
AIRS stands for Artificial Immune Recognition System
You are training an SVM classifier using a subset of training data (simulating an AIRS-like algorithm) and evaluating its performance on a test set.
Key Steps:
Data Generation: Random data (X) with 100 samples and 10 features, and random binary labels (y).
Data Splitting: The dataset is split into training and testing sets (80% training, 20% testing).
Scaling: The features are scaled using StandardScaler for better SVM performance.
Training the Model:
You randomly select detectors (20 data points) and use them to train an SVM with the RBF kernel.
The model is trained on this small subset of data.
Evaluation: The accuracy is calculated by comparing the model's predictions on the test set with the actual labels.
Code Flow:
train_airs: Selects a random subset of data (detectors), trains an SVM classifier, and returns the trained model.
Data Generation: Creates dummy data and labels, splits it into training and testing sets.
Scaling: Standardizes the data using StandardScaler.
Model Training and Evaluation: The model is trained using the subset, and accuracy is computed on the test set.
Output:
The accuracy of the model is printed as a percentage.'''

In [ ]:
#8 deap 
import random
from deap import base, creator, tools, algorithms

# Evaluation function (minimize a simple quadratic function)
def eval_func(individual):
    return sum(x**2 for x in individual),

# Setup DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -5, 5)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", eval_func)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create population and run GA
population = toolbox.population(n=50)
for _ in range(20):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = list(map(toolbox.evaluate, offspring))
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, len(population))

# Output best individual with rounded fitness value
best_ind = tools.selBest(population, 1)[0]
best_fitness = round(best_ind.fitness.values[0], 4)
print("Best individual:", best_ind)
print("Best fitness:", best_fitness)

'''Genetic Algorithm (GA) Overview:
A Genetic Algorithm (GA) is a search heuristic inspired by natural selection in biology. 
It is used to solve optimization problems by mimicking the process of evolution. 
The main idea is to evolve a population of potential solutions over generations, selecting the best solutions, 
combining them (crossover), and introducing small random changes (mutation) to create new, potentially better solutions.

Key Components:
Population: A group of individuals (solutions). Each individual represents a potential solution to the problem.
Fitness Function: A function that evaluates how good each individual solution is. In minimization problems, lower fitness values are better.
Selection: The process of choosing individuals based on their fitness to form the next generation. Typically, better solutions are more likely to be selected.
Crossover (Mating): Combining parts of two individuals to create offspring, allowing the best traits from both parents to be passed on.
Mutation: Randomly altering an individual’s attributes to introduce diversity and avoid local optima.
Termination: The algorithm stops after a set number of generations or when a satisfactory solution is found.'''

In [ ]:
#9 Ant colony
import numpy as np, random

dist = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

n = len(dist)
pher = np.ones((n, n))
ants, iters = 5, 100

def tour():
    t = [random.randint(0, n-1)]
    while len(t) < n:
        p = pher[t[-1]] / dist[t[-1]]
        p[[*t]] = 0
        p = p / p.sum()
        t.append(np.random.choice(range(n), p=p))
    return t

def length(t): return sum(dist[t[i]][t[(i+1)%n]] for i in range(n))

for _ in range(iters):
    paths = [tour() for _ in range(ants)]
    lens = [length(t) for t in paths]
    pher *= 0.5
    for t, l in zip(paths, lens):
        for i in range(n):
            pher[t[i]][t[(i+1)%n]] += 100 / l

best = min(paths, key=length)
print("Best tour:", best)
print("Length:", round(length(best), 2))

'''This code implements Ant Colony Optimization (ACO) to solve the Traveling Salesman Problem (TSP). The goal is to find the shortest route that visits all cities and returns to the starting point.
Key Steps:
Initialization: A distance matrix (dist) and pheromone matrix (pher) are initialized. Pheromones represent the desirability of each path.
Tour Creation: Each ant starts at a random city and selects the next city based on pheromone levels and distances. It continues until all cities are visited.
Pheromone Update: After each ant completes its tour, pheromones are updated. Paths with shorter tours get more pheromone, encouraging future ants to take those paths.
Iteration: The process repeats for a set number of iterations (100), with pheromone evaporation to prevent early convergence to suboptimal paths.
Output: After all iterations, the best tour and its total length are printed.'''

In [ ]:
#10 hotel management
#Client
import Pyro5.api
hotel = Pyro5.api.Proxy("PYRONAME:hotel.service")
while True:
    print("\n1. Book Room\n2. Cancel Booking\n3. Exit")
    choice = input("Choose: ")
    if choice == "1":
        name = input("Enter name: ")
        print(hotel.book(name))
    elif choice == "2":
        name = input("Enter name: ")
        print(hotel.cancel(name))
    elif choice == "3":
        break

#HotelRoom
#Server
import Pyro5.api
@Pyro5.api.expose
class Hotel:
    def __init__(self):
        self.bookings = {}

    def book(self, name):
        if name in self.bookings:
            return f"{name} already has a booking."
        self.bookings[name] = f"Room #{len(self.bookings) + 1}"
        return f"Room booked for {name}."

    def cancel(self, name):
        if name in self.bookings:
            del self.bookings[name]
            return f"Booking cancelled for {name}."
        return f"No booking found for {name}."

# Start server
daemon = Pyro5.api.Daemon()
uri = daemon.register(Hotel)
print("URI:", uri)

ns = Pyro5.api.locate_ns()
ns.register("hotel.service", uri)

print("Hotel Booking Server is running...")
daemon.requestLoop()

'''Client (Hotel Management Interface):
The client allows users to book or cancel hotel room bookings.
It communicates with the server using a Pyro5 proxy (a remote object).
The client provides three options:
Book Room: Requests room booking for a user by providing a name.
Cancel Booking: Cancels the booking for the user with the given name.
Exit: Ends the program.
Server (Hotel Room Booking Service):
The server defines the Hotel class, which manages room bookings in memory.
It has two main methods:
book(name): Books a room for a user if they don’t already have a booking.
cancel(name): Cancels the booking for a user if they have one.
The server runs in a Pyro5 daemon, which listens for incoming client requests.
The server is registered with the name server (hotel.service) so that clients can access it.
Flow:
The client communicates with the server to book or cancel a room.
The server manages bookings and responds to the client with appropriate messages.'''